# Day 2 - Data Exploration & Preparation (MIMIC-III Discharge summaries)

---

## 1. Dataset overview
**Files used:**
- `NOTEEVENTS.csv.gz` - contains all clinical notes from MIMIC-III (~2M rows)
- `DIAGNOSES_ICD.csv.gz` - contains ICD-9 dianosis codes per hospital admission (`HADM_ID`)

**Objective:**
Prepare a dataset for **multi-label text classification**:
> Given a discharge summary (`TEXT`). predict the list of ICD-9 codes (`LABELS`).

**Key Identifiers:**
| Field | Meaning |  
|--------|----------|  
| `SUBJECT_ID` | Patient identifier |  
| `HADM_ID` | Hospital admission ID |  
| `ICD9_CODE` | Diagnosis code |  
| `TEXT` | Discharge summary text |  

---

In [ ]:
# Environment info

import sys, platform
import pandas as pd
import numpy as np
from collections import Counter

print("Python:", sys.version)
print("Platform:", platform.platform())
print("pandas:", pd.__version__)
print("numpy:", np.__version__)

SEED = 42

In [ ]:
from pathlib import Path

ROOT = Path(".").cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent
DATA_RAW = ROOT / "data" / "raw"
DATA_PRO = ROOT / "data" / "processed"
REPORTS = ROOT / "reports"

NOTE_PATH = DATA_RAW / "NOTEEVENTS.csv.gz"
DIAG_PATH = DATA_RAW / "DIAGNOSES_ICD.csv.gz"
OUT_PATH = DATA_PRO / "notes_icd.csv"

print("NOTE_PATH:", NOTE_PATH)
print("DIAG_PATH:", DIAG_PATH)

## Step 1 - Inspect `NOTEEVENTS.csv.gz`
### Goal
Understand structure and identify which type of notes are relevant for ICD coding.

### Observations
- Columns: `['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT']`
- Key columns: `CATEGORY` → defines types of note (e.g., Nursing, Radiology, Discharge summary).

In [ ]:
df_notes_head = pd.read_csv(NOTE_PATH, nrows=1000, low_memory=False)
print("NOTEEVENTS (sample) shape:", df_notes_head.shape)
print("Columns:", df_notes_head.columns.to_list())
df_notes_head.head(3)

### CATEGORY Distribution

In [ ]:
df_cat  = pd.read_csv(NOTE_PATH, usecols=["CATEGORY"], low_memory=False)
cat_counts = df_cat["CATEGORY"].value_counts().reset_index()
cat_counts.columns =["CATEGORY", "count"]
cat_counts.head(10)

### Interpretation
- "**Discharge summary**" is the only note type that contains the *final diagnostic summary* for an admission.
- Other note types describe nursing procedures or imaging, not suitable for ICD prediction.
- **Decision:** Keep only notes where `CATEGORY == "Discharge summary"`.

---

## Step 2 - Inspect `DIAGNOSES_ICD.csv.gz`
## Goal
Understand structure and diversity of ICD-9 diagnosis codes.

### Observations
- Columns: `'ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'`
- One `HADM_ID` can have multiple `ICD9_CODE` entries.

In [ ]:
df_icd_head = pd.read_csv(DIAG_PATH, nrows=1000, low_memory=False)
print("DIAGNOSE_ICD sample shape:", df_icd_head.shape)
print("Columns", df_icd_head.columns.to_list())
df_icd_head.head(3)

### Summary Statistics
| Metric | Value | Description |  
|---------|--------|-------------|  
| Unique admission (`HADM_ID`) | ~58,976 | Number of unique hospital stays |  
| Unique ICD-9 codes | ~6,984 | Number of unique diagnosis codes |
| Mean codes/admission | ~11.0 | Average codes per hospital stay |
| Median codes/admission | ~9.0 | Indicates long-tail distribution |

In [ ]:
df_icd = pd.read_csv(DIAG_PATH, usecols=["HADM_ID", "ICD9_CODE"], low_memory=False)

n_adm = df_icd["HADM_ID"].nunique()
n_code = df_icd["ICD9_CODE"].nunique()
code_per_adm = df_icd.groupby("HADM_ID")["ICD9_CODE"].count()

print(f"Unique admission (HADM_ID): {n_adm}")
print(f"Unique ICD9 codes: {n_code}")
print(f"Mean #code/admission: {code_per_adm.mean():.2f}")
print(f"Median #code.admission: {code_per_adm.median():.2f}")

### Interpretation
- The dataset is inherently **multi-label** - each admission has several ICD codes.
- **Imbalance:** common diseases dominate (hypertension, heart failure), while rare codes are scarce.
- Future training requires **class reweighting or focal loss**.

---

## Step 3 - Group ICD Codes by Admission


### Goal
Aggregate all ICD-9 codes pere `HADM_ID` into a list of codes.

In [ ]:
df_icd_grouped = (
    df_icd.groupby("HADM_ID")["ICD9_CODE"]
    .apply(list)
    .reset_index()
    .rename(columns={"ICD9_CODE": "LABELS"})
)


print("Grouped ICD DataFrame shape:", df_icd_grouped.shape)
df_icd_grouped.head(5)

### Interpretation
- Each row represents a single hospital admission.
- The `LABELS` column contains the list of ICD-9 codes - the **ground truth** for the model.

---

## Step 4 - Filter Discharge Summaries
### Goal
Filter `NOTEEVENTS` to keep only discharge summaries

In [ ]:
# Filter discharge summaries only

df_notes = pd.read_csv(NOTE_PATH, usecols=["SUBJECT_ID", "HADM_ID", "CATEGORY", "TEXT"], low_memory=False)

df_notes = df_notes[df_notes["CATEGORY"] == "Discharge summary"].dropna(subset=["TEXT"])
print("Filtered notes shape:", df_notes.shape)
df_notes.head(10)

### Interpretation
This filtered subset contain only discharge summaries - each summarizing diagnoses and hospital course, making it ideal for ICD prediction.

---

## Step 5 - Merge Notes and ICD labels
### Goal
Combine `df_notes` (discharge summaries) with `df_icd_grouped` (ICD labels).

In [ ]:
# Merge discharge summaries with grouped ICD codes
df_merge = pd.merge(df_notes, df_icd_grouped, on="HADM_ID", how="inner")

print("Merged DataFrame shape:", df_merge.shape)
print("Columns:", df_merge.columns.to_list())
df_merge.head(10)

### Interpretation
The merged dataset forms the **core training data**, linking each discharge summary (`TEXT`) to its corresponding ICD labels (`LABELS`).

---

## Step 6 - Integrity checks & Label Statistics
### Integrity checks

In [ ]:
assert {"SUBJECT_ID", "HADM_ID", "TEXT", "LABELS"}.issubset(df_merge.columns)
assert df_merge["LABELS"].map(len).gt(0).all(), "Found admission without labels!"

print("Integrity check passed.")
print("Unique subjects:", df_merge["SUBJECT_ID"].nunique())
print("Unique admission:", df_merge["HADM_ID"].nunique())

### Results:
- No missing labels
- Each row = one hospital admission.
- `SUBJECT_ID` and `HADM_ID` are consistent.

### Top 20 ICD-9 Codes

In [ ]:
# [Summary] Save processed dataset
df_merge.to_csv(DATA_PRO / "note_icd.csv", index=False)

# [Analysis] Label frequency overview
freq = Counter([c for lst in df_merge["LABELS"] for c in lst])
top20 = pd.DataFrame(freq.most_common(20), columns=["ICD9_CODE", "count"])
top20.head(10)

| ICD9_CODE | count | Description|  
|---------|--------|-------------|  
| 4019 | 22,339 | Hypertension, unspecified |  
| 4280 | 14,931 | Congestive heart failure |  
| 42731 | 14,479 | Atrial fibrillation |  
| 41401 | 13,801 | Coronary atherosclerosis |  
| 4849 | 9,943 | Acute kidney failure |
| 25000 | 9,831 | Type II diabetes mellitus |  
| 2724 | 9,279 | Hyperlipidemia |  
| 51881 | 8,271 | Acute respiratory failure |  
| 5990 | 7,499 | Urinary tract infection |  
| 53081 | 6,738 | Exophageal reflux |  

### Interpretation
- The dataset shows strong **cardiometabolic dominance** - heart and metabolic diseases are most frequent.
- Label distribution is **long-tailed**: many rare codes appeaer only a few times.
- Future experiments should emphasize **Macro-F1** and **rare-code recall**.

---

## Research Reflections
### Key takeaways:
- *Discharge summaries* provide the best link between text and ICD codes.
- Dataset is large, multi-label, and heavily imbalanced.
- Grouping ICD codes per admission simplifies supervision and ensures cleaner labels.

### Next Steps:
1. Move preprocessing logic into `src/prepare_data.py`.
2. Perform subject-level split (train/val/test).
3. Train baseline models (TF-IDF + Logistic Regression).

### Safety note:
> Do **not** print raw text or PHI. Only analyze schema, counts, and label distributions.

---